<a href="https://colab.research.google.com/github/OvinduWijethunge/Next-Day-Rain-Predicts-In-Australia/blob/master/model_train%20(gboost).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Here I choose Gradient Boosting model as my ML model. i prefer to go with it. 
# i do not try multiple models here.

In [24]:
!nvidia-smi

Fri Jun  4 04:20:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 454.3MB 37kB/s 


In [25]:
import tensorflow as tf

In [26]:
tf.__version__

'2.5.0'

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
#df = pd.read_csv('./datasets/preprocessed.csv')
df = pd.read_csv('https://raw.githubusercontent.com/OvinduWijethunge/Next-Day-Rain-Predicts-In-Australia/master/datasets/preprocessed.csv')

In [29]:
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month,Day
0,22,8.694688,13.589273,0.6,2.263869,57.554920,0,8.870193,0,1,12.139245,14.131102,34.891151,13.142729,318.255596,1.375390,8.0,0.0,10.549671,13.043088,0,0,12,1
1,22,5.301024,14.668806,0.0,2.406140,69.201128,1,8.870193,4,2,3.174802,13.142729,23.417682,14.620089,319.018654,1.375395,7.0,1.0,10.705502,14.278149,0,0,12,2
2,22,8.423478,14.960437,0.0,2.496163,59.559201,2,9.074396,0,2,11.631293,15.105825,20.724608,17.019013,318.229277,1.375401,8.0,2.0,12.642979,13.737467,0,0,12,3
3,22,6.355600,16.068119,0.0,2.589230,77.023050,3,6.419909,12,14,7.376108,6.240251,23.860366,10.079368,320.859017,1.375428,0.0,5.0,11.170317,15.347521,0,0,12,4
4,22,10.860881,18.099591,1.0,2.819463,88.580965,0,8.553172,8,11,5.061140,12.139245,39.340953,18.425882,319.071265,1.375382,7.0,8.0,11.015816,16.877070,0,0,12,5


In [30]:
df.shape

(141626, 24)

In [31]:
df.isnull().mean()*100

Location         0.0
MinTemp          0.0
MaxTemp          0.0
Rainfall         0.0
Evaporation      0.0
Sunshine         0.0
WindGustDir      0.0
WindGustSpeed    0.0
WindDir9am       0.0
WindDir3pm       0.0
WindSpeed9am     0.0
WindSpeed3pm     0.0
Humidity9am      0.0
Humidity3pm      0.0
Pressure9am      0.0
Pressure3pm      0.0
Cloud9am         0.0
Cloud3pm         0.0
Temp9am          0.0
Temp3pm          0.0
RainToday        0.0
RainTomorrow     0.0
Month            0.0
Day              0.0
dtype: float64

In [32]:
y = df['RainTomorrow']
X = df.drop(['RainTomorrow'],axis=1)

In [33]:
X.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'Month', 'Day'],
      dtype='object')

In [34]:
y

0         0
1         0
2         0
3         0
4         0
         ..
141621    0
141622    0
141623    0
141624    0
141625    0
Name: RainTomorrow, Length: 141626, dtype: int64

In [35]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

os=RandomOverSampler(0.9)
X_ns,y_ns=os.fit_sample(X,y)
print("The number of classes before fit {}".format(Counter(y)))
print("The number of classes after fit {}".format(Counter(y_ns)))

X = X_ns
y = y_ns

The number of classes before fit Counter({0: 110382, 1: 31244})
The number of classes after fit Counter({0: 110382, 1: 99343})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

In [38]:
gb = GradientBoostingClassifier()

In [39]:
gb.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [40]:
grid={
    'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
    'n_estimators':[x for x in range(100,1000,2)],
    'max_depth':[2,3,4,5,6,7],
    'min_samples_split':[2,4,6,8,10,20,40,60,100], 
    'min_samples_leaf':[1,3,5,7,9],
    'max_features':[2,3,4,5,6,7],
    'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]
}

In [41]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True)

In [42]:
from sklearn.model_selection import RandomizedSearchCV
gb_random = RandomizedSearchCV(estimator=gb, param_distributions=grid, n_jobs=-1,n_iter = 50, cv=cv, scoring='accuracy',error_score=0,return_train_score=True,verbose=2)

In [43]:
gb_random.fit(X_train,y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 26.2min


KeyboardInterrupt: ignored

In [ ]:
best_model = gb_random.best_estimator_

In [ ]:
best_model

In [ ]:
gb_random.best_params_

In [ ]:
gb_random.best_score_

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
err  =(y_pred- y_test)

In [ ]:
err.value_counts().plot(kind='bar',color='purple')

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
indices=list(X_train)
feat_imp = pd.Series(best_model.feature_importances_, indices).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Importance of Features')
plt.ylabel('Feature Importance Score')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
print("accuracy is {}".format(best_model.score(X_test, y_test)))

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
baseline_roc_auc = roc_auc_score(y_test, best_model.predict(X_test))
fprB, tprB, thresholdsB = roc_curve(y_test, best_model.predict_proba(X_test)[:,1])

In [ ]:
plt.figure()
plt.plot(fprB, tprB, label='GB(area = %0.2f)' % baseline_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# ROC - AUC simple code
#metrics.plot_roc_curve(knn, X_test, y_test)
#metrics.roc_auc_score(y_test, y_pred4, average=None)

In [ ]:
### Predict test_y values and probabilities based on random forest model
probs_y=best_model.predict_proba(X_test) 
  # probs_y is a 2-D array of probability of being labeled as 0 (first column of array) vs 1 (2nd column in array)

from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, probs_y[:, 1]) 
   #retrieve probability of being 1(in second column of probs_y)
pr_auc = metrics.auc(recall, precision)

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[: -1], "b--", label="Precision")
plt.plot(thresholds, recall[: -1], "r--", label="Recall")
plt.ylabel("Precision, Recall")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

In [ ]:
import pickle
#open a file, where you ant to store the data
file = open('gboost.pkl', 'wb')
# dump information to that file
pickle.dump(best_model, file)
file.close()